## Анализ результатов A/B-тестирования

Компания интернет-магазина BitMotion Kit, в котором продаются геймифицированные товары для тех, кто ведёт здоровый образ жизни.

В будущем компания хочет расширить ассортимент товаров. Но перед этим нужно решить одну проблему. Интерфейс онлайн-магазина слишком сложен для пользователей — об этом говорят отзывы.

Чтобы привлечь новых клиентов и увеличить число продаж, владельцы магазина разработали новую версию сайта и протестировали его на части пользователей. По задумке, это решение доказуемо повысит количество пользователей, которые совершат покупку.

Необходимо провести оценку результатов A/B-теста.

### Техническое задание проведённого теста

Гипотеза компании заключается в следующем: упрощение интерфейса приведёт к тому, что в течение семи дней после регистрации в системе конверсия зарегистрированных пользователей в покупателей увеличится как минимум на три процентных пункта.

Параметры теста:

- название теста: `interface_eu_test`;
- группы: А (контрольная), B (новый интерфейс);
- дата набора новых пользователей: с 2020-12-01 по 2020-12-21 включительно;
- дата прохождения теста: с 2020-12-08 по 2020-12-29 включительно.


### Данные

- `https://code.s3.yandex.net/datasets/ab_test_participants.csv` — таблица участников тестов.
  Структура файла:

  - `user_id` — идентификатор пользователя;
  - `group` — группа пользователя;
  - `ab_test` — название теста;
  - `device` — устройство, с которого происходила регистрация.

- `https://code.s3.yandex.net/datasets/ab_test_events.zip` — архив с одним `csv`-файлом, в котором собраны события 2020 года;
  Структура файла:

  - `user_id` — идентификатор пользователя;
  - `event_dt` — дата и время события;
  - `event_name` — тип события;
  - `details` — дополнительные данные о событии.

**Цель исследования** - выяснить, повлиет ли обновленный интерфейс сайта на увеличение конверсии пользователей в покупателей. Ожидается, что в течение семи дней после регистрации конверсия увеличится на 3 процентных пункта.

Для проверки будет проведен Z-тест пропорций на двух группах пользователей.

In [1]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

In [2]:

URL = 'https://code.s3.yandex.net/datasets/'
participants = pd.read_csv(URL + 'ab_test_participants.csv')
events = pd.read_csv(URL + 'ab_test_events.zip', parse_dates=['event_dt'], dtype={'details': object})

In [3]:
display(participants.info())
display(events.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     787286 non-null  object        
 1   event_dt    787286 non-null  datetime64[ns]
 2   event_name  787286 non-null  object        
 3   details     249022 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.0+ MB


None

В таблице `participants` содержится **14 525** данных без пропусков. В таблице `events` содержится **787 286** данных, значительные пропуски присутствуют в столбце `details`.

In [4]:
display(participants.head())
display(events.head())

,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac


,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN



Проверим данные на наличие явных дубликатов:


In [5]:
display(participants.duplicated().sum())
display(events.duplicated().sum())

0

36318

В таблице `events` содержится 36 318 дубликатов, избавимся от них в следующих шагах.


Выделим пользователей, участвующих в тесте, и проверим:

   - соответствие требованиям технического задания,

   - равномерность распределения пользователей по группам теста,

   - отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

In [6]:
ab_test_participants = participants[participants['ab_test'] == 'interface_eu_test']

In [7]:
#Проверяем наличие тестовых групп
ab_test_participants['group'].value_counts()

B    5467
A    5383
Name: group, dtype: int64

В данных две группы (A и B) с примерно одинаковым количеством наблюдений

In [8]:
display(events[events['user_id'].isin(ab_test_participants['user_id'])]['event_dt'].min())
display(events[events['user_id'].isin(ab_test_participants['user_id'])]['event_dt'].max())

Timestamp('2020-12-06 14:10:01')

Timestamp('2020-12-30 18:34:08')

В данных отсутсвует период **2020-12-01 по 2020-12-05** и присутсвует лишний день **2020-12-30**. Кроме этого период сбора данных совпадает с техническим заданием.

Параметры теста совпадают с техническим заданием, за исключением несоответствия периодов сбора данных.

   
Проверим корректность периодов для регистрации и непосредственного сбора статистики:


In [9]:
reg_min = events[(events['user_id'].isin(ab_test_participants['user_id'])) & 
                 (events['event_name'] == 'registration')]['event_dt'].min()
reg_max = events[(events['user_id'].isin(ab_test_participants['user_id'])) & 
                 (events['event_name'] == 'registration')]['event_dt'].max()

exp_min = events[(events['user_id'].isin(ab_test_participants['user_id'])) & 
                 (events['event_name'] != 'registration')]['event_dt'].min()
exp_max = events[(events['user_id'].isin(ab_test_participants['user_id'])) & 
                 (events['event_name'] != 'registration')]['event_dt'].max()

print(f'Период регистрации пользователей: с {reg_min} по {reg_max}')
print(f'Период сбора статистики: с {exp_min} по {exp_max}')

Период регистрации пользователей: с 2020-12-06 14:10:01 по 2020-12-24 21:57:10
Период сбора статистики: с 2020-12-07 00:04:00 по 2020-12-30 18:34:08



В данных присутствуют пользователи, зарегистрированные после 21 числа. А в периоде сбора данных присутствуют наблюдения от 7 числа.

In [10]:
test_groups = participants.groupby('user_id')['ab_test'].nunique().reset_index()
test_groups_more_than_one = test_groups[test_groups['ab_test'] > 1]
display(test_groups_more_than_one.shape[0])

887

В выборке содержится 887 пользователей, принимавших участие в более чем двух тестах

In [11]:
participants[participants['user_id'].isin(test_groups_more_than_one['user_id'])]['ab_test'].unique()

array(['recommender_system_test', 'interface_eu_test'], dtype=object)

Пересечения в том числе присутствуют для теста `interface_eu_test`, исследуемого в работе. Проверим также наличие пересечения контрольной и тестовых групп:

In [12]:
test_groups_A_B = ab_test_participants.groupby('user_id')['group'].nunique().reset_index()
test_groups_A_B_more_than_one = test_groups_A_B[test_groups_A_B['group'] > 1]
display(test_groups_A_B_more_than_one.shape[0])

0

В данных нет пересечений по тестовым группам A и B, удалим из данных пользователей, участвовавших сразу в нескольких экспериментах:

In [13]:
ab_test_participants_cleaned = ab_test_participants[~ab_test_participants['user_id'].isin(test_groups_more_than_one['user_id'])]

In [14]:
ab_test_participants_cleaned['group'].value_counts()

B    5011
A    4952
Name: group, dtype: int64

  
Распределение данных между группами осталось равномерным.


Оценим достаточность выборки для получения статистически значимых результатов A/B-теста. Заданные компанией параметры: 

- базовый показатель конверсии — 30%,

- мощность теста — 80%,

- достоверность теста — 95%.

In [15]:
ab_test_participants_events = ab_test_participants_cleaned.merge(events, on = 'user_id', how = 'inner')

In [16]:
A_group = ab_test_participants_events[ab_test_participants_events['group'] == 'A']
B_group = ab_test_participants_events[ab_test_participants_events['group'] == 'B']

# Количество покупок
A_purchase = A_group[A_group['event_name'] == 'purchase'].shape[0]
B_purchase = B_group[B_group['event_name'] == 'purchase'].shape[0]

# Доля успешных сессий
A_purchase_share = A_purchase / A_group.shape[0]
B_purchase_share = B_purchase / B_group.shape[0]

Проверим достаточность выборок:

In [17]:
if (A_purchase_share*A_group.shape[0] > 10)and (B_purchase_share*B_group.shape[0] > 10):
    print('Предпосылка о достаточном количестве данных выполняется!')
else:
    print('Предпосылка о достаточном количестве данных НЕ выполняется!')

Предпосылка о достаточном количестве данных выполняется!


   
Также при использовании калькулятора Эвана Миллера с минимальным детектируемым эффектом в 3 п.п. необходимый размер одной выборки составил 3 692 наблюдения, что также подтверждает достаточность выборки в нашем эксперименте.


Проанализируем данные о пользовательской активности по таблице `ab_test_events`:


In [18]:
ab_test_events = events[events['user_id'].isin(ab_test_participants_cleaned['user_id'])]

In [19]:
ab_test_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73815 entries, 64672 to 780371
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     73815 non-null  object        
 1   event_dt    73815 non-null  datetime64[ns]
 2   event_name  73815 non-null  object        
 3   details     19450 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.8+ MB


In [20]:
# Проверим наличие дубликатов
ab_test_events.duplicated().sum()

5741

In [21]:
# Отчистим данные от дубликатов
ab_test_events_cleaned = ab_test_events.drop_duplicates(keep = 'first', inplace=False)

In [22]:
ab_test_events_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68074 entries, 64672 to 780371
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     68074 non-null  object        
 1   event_dt    68074 non-null  datetime64[ns]
 2   event_name  68074 non-null  object        
 3   details     18864 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.6+ MB


Рассчитаем время (лайфтайм) совершения события пользователем после регистрации и оставим только те события, которые были выполнены в течение первых семи дней с момента регистрации;

In [23]:
# Выбираем даты регистрации по техническому заданию
registrations = ab_test_events_cleaned[(ab_test_events_cleaned['event_name'] == 'registration') & 
                                       (ab_test_events_cleaned['event_dt'] >= '2020-12-01') & 
                                       (ab_test_events_cleaned['event_dt'] <= '2020-12-21')][['user_id', 'event_dt']]
registrations.rename(columns={'event_dt': 'registration_date'}, inplace=True)

# Добавления даты регистрации к данным
ab_test_events_cleaned = ab_test_events_cleaned.merge(registrations, on='user_id', how='left')

# Оставляем данные в течение 7 дней после момента регистрации, саму регистрацию в данные не включаем
ab_test_events_filtered = ab_test_events_cleaned[
             (ab_test_events_cleaned['event_dt'] <= ab_test_events_cleaned['registration_date'] + pd.Timedelta(days=7)) &
             (ab_test_events_cleaned['event_name'] != 'registration')]
display(ab_test_events_filtered.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39242 entries, 15 to 65190
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            39242 non-null  object        
 1   event_dt           39242 non-null  datetime64[ns]
 2   event_name         39242 non-null  object        
 3   details            4515 non-null   object        
 4   registration_date  39242 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 1.8+ MB


None

In [24]:
display(registrations.shape[0])
display(registrations['user_id'].nunique())

7962

7962


В данных о регистрации каждому пользователю соответствует одна запись о регистрации.


Рассчитаем для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [25]:
# Присоединяем информацию о тестовых группах
ab_test_events_final = ab_test_events_filtered.merge(ab_test_participants_cleaned[['user_id','group']], 
                                                     on = 'user_id', how = 'inner')
ab_test_events_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39242 entries, 0 to 39241
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            39242 non-null  object        
 1   event_dt           39242 non-null  datetime64[ns]
 2   event_name         39242 non-null  object        
 3   details            4515 non-null   object        
 4   registration_date  39242 non-null  datetime64[ns]
 5   group              39242 non-null  object        
dtypes: datetime64[ns](2), object(4)
memory usage: 2.1+ MB


- Сделаем предварительный общий вывод об изменении пользовательской активности в тестовой группе по сравнению с контрольной.


Создадим булевую переменную, на основе которой посчитаем количество покупок:


In [28]:
ab_test_events_final['purchase_made'] = 0 
# переменная принимает значение 0 когда целевое действие не связано с покупкой

ab_test_events_final.loc[ab_test_events_final['event_name'] == 'purchase', 'purchase_made'] = 1 
# принимает значение 1 когда пользователь совершил покупку

In [29]:
# Количество покупок уникальных пользователей
A_purchase_count = ab_test_events_final[(ab_test_events_final['group'] == 'A') & 
                                        (ab_test_events_final['purchase_made'] == 1)]['user_id'].nunique()

B_purchase_count = ab_test_events_final[(ab_test_events_final['group'] == 'B') & 
                                        (ab_test_events_final['purchase_made'] == 1)]['user_id'].nunique()

# Общее количество уникальных пользователей
A_all = ab_test_events_final[ab_test_events_final['group'] == 'A']['user_id'].nunique()
B_all = ab_test_events_final[ab_test_events_final['group'] == 'B']['user_id'].nunique()

# Конверсия в покупку
A_conv = round(A_purchase_count / A_all, 4)
B_conv = round(B_purchase_count / B_all, 4)

print(f'Общее количество уникальных пользователей в выборке А: {A_all}, в выборке B: {B_all}')
print(f'Количество покупок уникальных пользователей в выборке А: {A_purchase_count}, в выборке B: {B_purchase_count}')

Общее количество уникальных пользователей в выборке А: 3936, в выборке B: 4026
Количество покупок уникальных пользователей в выборке А: 1053, в выборке B: 1146


In [30]:
print(f'Конверсия в покупку в выборке А: {A_conv}, в выборке B: {B_conv}')
print(f'Разница в конверсии между тестовой и контрольной выборками составила: {round(B_conv - A_conv, 3)}')

Конверсия в покупку в выборке А: 0.2675, в выборке B: 0.2846
Разница в конверсии между тестовой и контрольной выборками составила: 0.017


4. оценка результатов A/B-тестирования:

- Проверим изменение конверсии статистическим тестом.

In [31]:
alpha = 0.05

stat_ztest, p_value_ztest = proportions_ztest(
[A_purchase_count, B_purchase_count],
[A_all, B_all],
alternative='smaller'
)

if p_value_ztest > alpha:
    print(f'pvalue={p_value_ztest} > {alpha}')
    print('Не получилось отвергнуть Нулевую гипотезу')
else:
    print(f'pvalue={p_value_ztest} < {alpha}')
    print('Нулевая гипотеза не находит подтверждения')

pvalue=0.04380487624266547 < 0.05
Нулевая гипотеза не находит подтверждения


- Опишите выводы по проведённой оценке результатов A/B-тестирования. Что можно сказать про результаты A/B-тестирования? Был ли достигнут ожидаемый эффект в изменении конверсии?


В результате работы был проведен Z-тест пропорций с уровнем значимости 5%. P-value получился меньше заданного уровня значимости, поэтому мы можем говорить о статистически значимом изменении конверсии пользователей в покупателей после обновления интерфейса приложения.

Однако предположение о том, что изменение интерфейса увеличит конверсию не менее чем на 3 процентных пункта не подтвердилось, так как разница средних между двумя выборками составила 1.7 процентных пункта.
